# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data_to_load = "../output_data/cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
cleaned_weather_data = weather_data.dropna()
del cleaned_weather_data["City_ID"]
cleaned_weather_data['City'] = cleaned_weather_data['City'].str.capitalize()
cleaned_weather_data.head(20)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,Mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,Qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,Zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,Cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,Atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,Flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
8,Tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,Pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
10,Aykhal,90,RU,1558378756,48,65.95,111.51,39.20,11.18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

locations = cleaned_weather_data[["Lat", "Lng"]]
humidity = cleaned_weather_data["Humidity"].astype(float)

In [7]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# cloudiness less than 20
ideal_cloudiness = dropna_weather_data[cleaned_weather_data["Cloudiness"] < 20]
# max temp above 70 degrees F
ideal_maxtemp_bottom = ideal_cloudiness[ideal_cloudiness["Max Temp"] > 70]
# max temp below 80 degrees F
ideal_maxtemp_top = ideal_maxtemp_bottom[ideal_maxtemp_bottom["Max Temp"] < 80]
# wind speed below 10 mph
ideal_windspeed = ideal_maxtemp_top[ideal_maxtemp_top["Wind Speed"] < 10]
# humidity below 60 %
ideal_vacationspot = ideal_windspeed[ideal_windspeed["Humidity"] < 50]
# Set Index
ideal_vacationspot_indexed = ideal_vacationspot.reset_index()
del ideal_vacationspot_indexed["index"]
ideal_vacationspot_indexed

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93
1,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17


In [10]:
vacation_locations = ideal_vacationspot_indexed[["Lat", "Lng"]]
vacation_humidity = ideal_vacationspot_indexed["Humidity"].astype(float)
vacation_fig = gmaps.figure()
vacation_heat_layer = gmaps.heatmap_layer(vacation_locations, weights=vacation_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

vacation_fig.add_layer(vacation_heat_layer)
vacation_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotels = []
for city in range(len(ideal_vacationspot_indexed["City"])):

    lat = ideal_vacationspot_indexed.loc[city]["Lat"]
    lng = ideal_vacationspot_indexed.loc[city]["Lng"]
    city_coords = f"{lat},{lng}"
    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("No Nearest Hotel Found")
ideal_vacationspot_indexed["Nearest Hotel"] = hotels
ideal_vacationspot_indexed

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Nearest Hotel
0,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93,Hôtel La Giralda
1,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17,تاکسی سرویس آرامش


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_vacationspot_indexed.iterrows()]
locations = ideal_vacationspot_indexed[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vacation_fig.add_layer(markers)

# Display Map
vacation_fig


Figure(layout=FigureLayout(height='420px'))